# This notebook allow users to pull NOAA Coastal Ocean ReAnalysis (CORA) data that is stored on NOAA's Open Data Dissemination (NODD) service. Data is extracted at the nearest model node to user provided geographic coordinates and the data is displayed in a time series plot.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask
import intake
import xarray as xr
import holoviews as hv
import geoviews as gv
import hvplot.xarray
import holoviews.operation.datashader as dshade
from bokeh.models import DatetimeTickFormatter, HoverTool
import cmocean

hv.extension('bokeh')

**Access the data on the NODD and initialize the available CORA datasets.** 

*This accesses a .yml file located on the NODD that shows which CORA output files are available to import.*

In [ ]:

catalog = intake.open_catalog("s3://noaa-nos-cora-pds/CORA_intake.yml",storage_options={'anon':True})
list(catalog)


**The CORA-V1-fort.63 file contains hourly water levels at the model mesh nodes for 1979-2022. The CORA-V1-maxele.63 file contains the maximum water level for the entire 1979-2022 period modeled at each of the model nodes. The CORA-V1-fort.64 file contains the hourly current velocities (u and v) at each of the model nodes for 1979-2022. The CORA-V1-500m-grid-1979-2022 file contains hourly water levels that have been interpolated from the model mesh nodes to uniformly space 500-meter grid nodes.**

## Using to_dask() with one of the catalog files will create an xarray dataset that contains the model grid information along with the time and water level ('zeta') variables.

**This is the dataset for the hourly water levels at all model nodes.**

In [ ]:
ds = catalog["CORA-V1-fort.63"].to_dask()
ds

## Create function to find the nearest node to a given set of coordinates.

In [ ]:
# find the indices of the points in (x,y) closest to the points in (xi,yi)
def nearxy(x,y,xi,yi):
    ind = np.ones(len(xi),dtype=int)
    for i in range(len(xi)):
        dist = np.sqrt((x-xi[i])**2+(y-yi[i])**2)
        ind[i] = dist.argmin()
    return ind

**Provide coordinates.**

In [ ]:

lat = 32.751
lon = -79.867

**Run the function.**

In [ ]:
ind = nearxy(ds['x'].values,ds['y'].values,[lon], [lat])

**Extract data for specified time and create plot using hvplot.**

In [ ]:
start="2018-09-13"
end="2018-09-15"

tickfmt = DatetimeTickFormatter(years="%m/%d/%Y", months="%m/%d/%Y", days = '%m/%d/%Y', minsec = ':%M:%S')
tooltips = [
    ("time", "@time{%F %T}"),
    ("water level", "@zeta"),
]
hover = HoverTool(tooltips=tooltips,formatters={
        '@time': 'datetime'})

ds['zeta'][:,ind].sel(time=slice(start, end)).hvplot(x='time', grid=True, xformatter=tickfmt, tools=[hover])